# Utilizing Llama 3.1 405B for Summarizing and Preparing Instruction Fine-Tuned Dataset

In this notebook, we will walk you through the process of utilizing a larger language model (LLM) like Llama 3.1 405B, Meta AI's latest and most advanced model, to create a dataset for instruction fine-tuning. This dataset will be used to perform distillation by fine-tuning a smaller model, such as Llama 3 8B.

By leveraging the capabilities of Llama 3.1 405B, we can generate high-quality, concise training data that enhances the performance of smaller models. This approach is particularly useful for tasks that require detailed and specific instructions.

Before we begin, ensure that you have access to Llama 3.1 405B, which is now available on Amazon SageMaker Jumpstart. You can find the dataset we will be using [here](https://huggingface.co/datasets/deepmind/aqua_rat).

You can run the notebook on an Amazon SageMaker Studio notebook, or a SageMaker notebook instance without manually setting your aws credentials.

Let's get started!

### Amazon SageMaker JumpStart

![Alt text](imgs/jumpstart-overview-img1.png "SageMaker JumpStart Overview")

**Amazon SageMaker JumpStart** is a powerful feature within Amazon SageMaker designed to help you quickly get started with LLMs by providing access to a wide range of pre-trained foundation models (FM). We'll be using this for deploying and fine tuning our models.

Key Features
- **Pre-trained Models**: SageMaker JumpStart provides a variety of pre-trained models from different model providers (Llama, Mistral, Cohere, Stablity) for different problem types, enabling you to start your machine learning projects without the need to build models from scratch.

- **Training and Tuning**: With a few clicks, you can train and fine-tune these models to better fit your specific data and use case before deploying them.

- **Solution Templates**: JumpStart offers solution templates that automatically set up the necessary infrastructure for common use cases, streamlining the deployment process.

### Llama 3.1 405B Model

Llama 3.1 405B is the largest model in the family of Llama 3.1 models. Llama 3.1 model family is a collection of pre-trained and instruction-tuned LLMs which already includes 8B and 70B parameter sizes. Llama 3.1 405B comes with new capabilities including multi-language support and a 128k context window. These models are stronger overall capabilities and are ideal for content creation, conversational AI, language understanding, research and development (R&D), and enterprise applications.


### Llama 3 8B Model

LLama 3 8B is an LLM with 8 billion parameters designed to deliver high performance across a variety of tasks while maintaining cost efficiency. This model is particularly advantageous for developers and organizations looking to implement advanced AI capabilities without the need for extensive computational resources. LLaMA 3 8B is optimized for dialogue and other interactive applications, demonstrating strong performance in benchmarks such as MMLU, AGIEval, and CommonSenseQA, where it outperforms many open-source models of similar size. Its ability to run on more affordable hardware highlights its potential for cost-effective deployment in real-time applications like chatbots and customer support systems.



### Prequisites
 In order to follow along in this notebook, you'll need access to the following:

 - An AWS account with SageMaker endpoint capacity for an ml.p4de instance type. You can find more information about how to request a service limit increase [here](https://docs.aws.amazon.com/servicequotas/latest/userguide/request-quota-increase.html).

 - An [AWS Identity and Access Management (IAM)](https://aws.amazon.com/iam/) role to access SageMaker. To learn more about how IAM works with SageMaker, refer to [Identity and Access Management for Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html).
 
 - Access to SageMaker Studio or a SageMaker notebook instance or an interactive development environment (IDE) such as PyCharm or Visual Studio Code. We recommend using SageMaker Studio for straightforward deployment and inference.


### In this notebook, we perform the following high level steps: 

1. We deploy a `Llama3-8b instruct` model and generate inferences on a `deepmind/aqua_rat` dataset.

1. Deploy and leverage the capabilities of the new `Llama 3.1 405B Model` to generate labels and corresponding data to be used to do distillation by fine-tuning`Llama3-8b instruct`

1. Test the fine-tuned `Llama3-8b instruct` model and test the model against the same questions to showcase the increase in response quality.

In [1]:
# Import necessary libraries
import logging
import sagemaker
from sagemaker import get_execution_role
from sagemaker.jumpstart.model import JumpStartModel
import json
from IPython.core.display import display, HTML
 
from botocore.exceptions import ClientError
import os
from botocore.config import Config


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


/tmp/ipykernel_8434/2991053635.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# We'll use this function to call inference on our deployed models
def run_inference(predictor, example_payloads):
    for payload in example_payloads:
        response = predictor.predict(payload)
        response = response[0] if isinstance(response, list) else response
        print("Input:\n", payload["inputs"], end="\n\n")
        print("Output:\n", response["generated_text"].strip(), end="\n\n\n")


## Dataset Exploration and Preparation

In this section, we will explore a dataset from the Hugging Face Hub using the HF Datasets library. Hugging Face provides a vast collection of datasets for various tasks in natural language processing (NLP), computer vision, and audio processing. This exploration will help us understand the structure, features, and contents of the dataset, enabling us to prepare it for training and evaluation in our machine learning models. The [deepmind/aqua_rat](https://huggingface.co/datasets/deepmind/aqua_rat) dataset is a large-scale collection of approximately 100,000 algebraic word problems, each accompanied by a detailed natural language rationale explaining the solution process. This dataset is designed to train and evaluate models that not only generate the correct answer but also provide a step-by-step explanation, making it ideal for tasks requiring mathematical reasoning and natural language understanding.

In [5]:
!pip install datasets

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 54.7 MB/s eta 0:00:00
Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 37.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.0
    Uninstalling fsspec-2024.6.0:
      Successfully uninstalled fsspec-

In [6]:
from datasets import load_dataset, DatasetDict

# Load the dataset
dataset_name = "CShorten/CDC-COVID-FAQ"
dataset = load_dataset(dataset_name)

# Split the train dataset into train and test splits
split_ratio = 0.8  # 80% for training, 20% for testing
train_test_split = dataset['train'].train_test_split(test_size=1 - split_ratio)

# Optionally split the train set further into train and validation sets
train_validation_split = train_test_split['train'].train_test_split(test_size=0.1)  # 10% for validation

# Create a new dataset dictionary with the new splits
dataset = DatasetDict({
    'train': train_validation_split['train'],
    'validation': train_validation_split['test'],
    'test': train_test_split['test']
})

# Display the number of examples in each split after splitting
print("\nNumber of Examples in Each Split After Splitting:")
for split in dataset.keys():
    if dataset[split] is not None:
        print(f"{split}: {len(dataset[split])} examples")

[2024-08-06 06:59:03,859] p8434 {config.py:58} INFO - PyTorch version 2.2.0 available.



Number of Examples in Each Split After Splitting:
train: 43 examples
validation: 5 examples
test: 13 examples


In [15]:
# Import the necessary functions from the datasets library with test split
from datasets import load_dataset, DatasetDict

# Load the deepmind/aqua_rat dataset from the Hugging Face Hub
dataset_name = "CShorten/CDC-COVID-FAQ"
dataset = load_dataset(dataset_name)

# Display basic information about the dataset
print(f"Dataset: {dataset_name}")
print(dataset)

# Display the dataset's features
print("\nDataset Features:")
print(dataset['train'].features)

# Display a few examples from the dataset
print("\nSample Examples:")
for i in range(3):
    print(dataset['train'][i])

# Display the number of examples in each split
print("\nNumber of Examples in Each Split:")
for split in dataset.keys():
    print(f"{split}: {len(dataset[split])} examples")

# Split the train dataset into train and test splits
split_ratio = 0.8  # 80% for training, 20% for testing
train_test_split = dataset['train'].train_test_split(test_size=1 - split_ratio)

# Optionally split the train set further into train and validation sets
train_validation_split = train_test_split['train'].train_test_split(test_size=0.1)  # 10% for validation

# Create a new dataset dictionary with the new splits
dataset = DatasetDict({
    'train': train_validation_split['train'],
    'validation': train_validation_split['test'],
    'test': train_test_split['test']
})

# Display the number of examples in each split after splitting
print("\nNumber of Examples in Each Split After Splitting:")
for split in dataset.keys():
    if dataset[split] is not None:
        print(f"{split}: {len(dataset[split])} examples")

# Extract 20 questions from the train split
questions = dataset['train'].select(range(20))['question']

# Display the first 20 questions
print("\nFirst 20 Questions:")
for i, question in enumerate(questions):
    print(f"{i+1}: {question}")

Dataset: CShorten/CDC-COVID-FAQ
DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'category', 'question', 'answer'],
        num_rows: 61
    })
})

Dataset Features:
{'Unnamed: 0': Value(dtype='int64', id=None), 'category': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None)}

Sample Examples:
{'Unnamed: 0': 0, 'category': 'COVID-19 Risk', 'question': 'Who is at risk for infection with SARS-CoV-2, the virus that causes COVID-19?', 'answer': 'Currently, those at greatest risk of infection are persons who have had prolonged, unprotected close contact (i.e., within 6 feet for 15 minutes or longer) with a patient with confirmed SARS-CoV-2 infection, regardless of whether the patient has symptoms. Persons frequently in congregate settings (e.g., homeless shelters, assisted living facilities, college or university dormitories) are at increased risk of acquiring infection because of the increased likelihood of

### Deploying Llama 3 8B Instruct

In this section, we will deploy the base, pre-trained LLama 3 8B model and test it against a subset of our dataset to evaluate its responses compared to the larger LLama 3.1 405B model. Initially, we expect the smaller model to produce lower-quality responses. By identifying these deficiencies, we can generate high-quality synthetic data using the 405B model and subsequently do distillation by fine-tuning the 8B model. This process aims to demonstrate the improvement in response quality after fine-tuning the 8B model with the generated dataset.

> You'll need a `g5.12xlarge` instance for endpoint usage to deploy this model.

In [8]:
# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Specify the role ARN directly
role = get_execution_role(sagemaker_session=sagemaker_session)

# Select a model ID and version
llama_3_8b_model_id = "meta-textgeneration-llama-3-8b-instruct" # Replace with your chosen model ID

# If your selected model is gated, you will need to set accept_eula to True to accept the model end-user license agreement (EULA).
accept_eula = True

# Deploy the model to a SageMaker endpoint
llama_3_8b_model = JumpStartModel(model_id=llama_3_8b_model_id,role=role)
llama_3_8b_predictor = llama_3_8b_model.deploy(accept_eula=accept_eula)

# example_payloads = llama_3_8b_model.retrieve_all_examples() # uncomment if you want to preloaded examples instead

question = questions[0]

example_payloads = [
    {
        "inputs": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "parameters": {
            "max_new_tokens": 1024,
            "top_p": 0.9,
            "temperature": 0.6,
            "details": True,
            "stop": "<|eot_id|>"
        }
    }
]

print("Running inference with LLama 3 8B model:\n")
run_inference(llama_3_8b_predictor, example_payloads)


[2024-08-06 06:59:17,806] p8434 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
[2024-08-06 06:59:17,964] p8434 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
[2024-08-06 06:59:18,430] p8434 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
Model 'meta-textgeneration-llama-3-8b-instruct' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-west-2.s3.us-west-2.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
[2024-08-06 06:59:18,987] p8434 {utils.py:566} INFO - Model 'meta-textgeneration-llama-3-8b-instruct' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-west-2.s3.us-west-2.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
Using model 'meta-textgeneration-llama-3-8b-instruct' with wildcard version identifier '*'. You can pin to versi

-------------!Running inference with LLama 3 8B model:

Input:
 <|begin_of_text|><|start_header_id|>user<|end_header_id|>

Can employees choose to wear respirators when not required by the employer?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Output:
 In general, employees are not required to wear respirators unless their employer has a legitimate reason to require them to do so. However, in some cases, employees may choose to wear respirators even if not required by their employer. Here are some scenarios:

1. **Employee's personal choice**: An employee may choose to wear a respirator as a personal precaution, even if not required by their employer, if they are concerned about their health or the health of others in the workplace.
2. **Employee's medical condition**: An employee with a medical condition, such as a respiratory disease, may choose to wear a respirator as a precautionary measure to protect themselves from exposure to airborne contaminants or allergens.
3. *

## Deploying LLama 3.1 405B Instruct

In this section, we will deploy the LLama 3.1 405B model to compare its responses with those of the smaller LLama 3 8B model. This deployment will allow us to evaluate the performance differences and identify areas where the 8B model's responses can be improved. By analyzing the responses from the 405B model, we can generate high-quality data for distillation of the 8B model, enhancing its accuracy and effectiveness for domain-specific tasks.

> You'll need a 'p5.48xlarge' instance for endpoint usage to deploy this model.

In [9]:
# Select a model ID and version
llama_3_1_405b_model_id = "meta-textgeneration-llama-3-1-405b-instruct-fp8" # Replace with your chosen model ID

# If your selected model is gated, you will need to set accept_eula to True to accept the model end-user license agreement (EULA).
accept_eula = True

# Deploy the model to a SageMaker endpoint
llama_3_1_405b_model = JumpStartModel(model_id=llama_3_1_405b_model_id,role=role)
llama_3_1_405b_predictor = llama_3_1_405b_model.deploy(accept_eula=accept_eula)

# example_payloads = model.retrieve_all_examples()

question = questions[0]

example_payloads = [
    {
        "inputs": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "parameters": {
            "max_new_tokens": 1024,
            "top_p": 0.9,
            "temperature": 0.6,
            "details": True
        }
    }
]


# Test the deployed endpoint
print("Running inference with LLama 3.1 405B model:\n")
run_inference(llama_3_1_405b_predictor, example_payloads)

Model 'meta-textgeneration-llama-3-1-405b-instruct-fp8' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-west-2.s3.us-west-2.amazonaws.com/fmhMetadata/eula/llama3_1Eula.txt for terms of use.
[2024-08-06 07:06:38,786] p8434 {utils.py:566} INFO - Model 'meta-textgeneration-llama-3-1-405b-instruct-fp8' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-west-2.s3.us-west-2.amazonaws.com/fmhMetadata/eula/llama3_1Eula.txt for terms of use.
Using model 'meta-textgeneration-llama-3-1-405b-instruct-fp8' with wildcard version identifier '*'. You can pin to version '2.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
[2024-08-06 07:06:38,789] p8434 {cache.py:619} WARNING - Using model 'meta-textgeneration-llama-3-1-405b-instruct-fp8' with wildcard version identifier '*'. You can pin to version '2.0.0' for more stable results. Note that models may h

-------------------------------------!Running inference with LLama 3.1 405B model:

Input:
 <|begin_of_text|><|start_header_id|>user<|end_header_id|>

Can employees choose to wear respirators when not required by the employer?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Output:
 Yes, employees can choose to wear respirators even when not required by the employer, but there are some conditions and considerations that apply. Here are the details:

**OSHA Regulations:**

The Occupational Safety and Health Administration (OSHA) allows employees to wear respirators voluntarily, as long as the employer permits it and the respirator use does not interfere with the employee's work duties or create a hazard. (29 CFR 1910.134(c)(2))

**Voluntary Use of Respirators:**

When an employee chooses to wear a respirator voluntarily, the employer is not required to provide the respirator or pay for it. However, the employer must still ensure that the employee is properly trained on the use

## Using Llama 3.1 405B for Data Labeling/Generation

In this section, we will leverage the LLama 3.1 405B model to generate high-quality synthetic data for distillation by fine-tuning the LLama 3 8B model. By using the 405B model to generate responses to domain-specific prompts, we can create a labeled dataset that will be used to fine-tune the 8B model, improving its accuracy and effectiveness in specific tasks.

In [10]:
# Load the dataset and select the training data
dataset = load_dataset('CShorten/CDC-COVID-FAQ', split='train')
questions = dataset.select(range(48))['question']

# Function to run inference and generate synthetic data using SageMaker JumpStart
def generate_synthetic_data(predictor, questions):
    synthetic_data = []
    for question in questions:
        # Add Chain of Thought Reasoning prompt to the question
        user_message = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        payload = {
            "inputs": user_message,
            "parameters": {
                "max_new_tokens": 512,
                "top_p": 0.9,
                "temperature": 0.0
            }
        }
        try:
            # Send the message to the model
            response = predictor.predict(payload)
            # print(f"Response: {response}")  # Debugging statement to inspect the response structure
            
            # Directly handle the response without JSON parsing
            if isinstance(response, list) and 'generated_text' in response[0]:
                response_text = response[0]['generated_text'].strip()
            else:
                response_text = response['generated_text'].strip()
            
            synthetic_data.append({
                "instruction": question,
                "response": response_text
            })
        except (ClientError, Exception) as e:
            print(f"ERROR: Reason: {e}")
            break 

    return synthetic_data

# Generate synthetic data using the SageMaker JumpStart deployed model
synthetic_data = generate_synthetic_data(llama_3_1_405b_predictor, questions)

# Save the synthetic data to a JSONL file
with open('synthetic_data_gen_faq_file.jsonl', 'w') as f:
    for entry in synthetic_data:
        f.write(json.dumps(entry) + '\n')

## (Optional) Bedrock Example

> Note: You'll probably need [Provisioned Throughput](https://docs.aws.amazon.com/bedrock/latest/userguide/prov-throughput.html)

In [ ]:
# Initialize the Bedrock client
config = Config(read_timeout=5000)
client = boto3.client("bedrock-runtime", region_name="us-west-2", config=config)

# Set the model ID, e.g., Llama 3.1 405b.
model_id = "meta.llama3-1-405b-instruct-v1:0"

# Load the dataset and select the first 20 questions
dataset = load_dataset('deepmind/aqua_rat', split='train')
questions = dataset.select(range(2000))['question']

# Function to run inference and generate synthetic data using Bedrock
def generate_synthetic_data(client, model_id, questions):
    synthetic_data = []
    for question in questions:
        # Add Chain of Thought Reasoning prompt to the question
        user_message = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        conversation = [
            {
                "role": "user",
                "content": [{"text": user_message}],
            }
        ]
        try:
            # Send the message to the model, using a basic inference configuration.
            response = client.converse(
                modelId=model_id,
                messages=conversation,
                inferenceConfig={
                    "maxTokens": 1024,
                    "temperature": 0.0,
                    "topP": 0.9
                },
            )

            # Extract the response text
            response_text = response["output"]["message"]["content"][0]["text"].strip()
            synthetic_data.append({
                "instruction": question,
                "response": response_text
            })
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
            break

    return synthetic_data

# Generate synthetic data using Bedrock
synthetic_data = generate_synthetic_data(client, model_id, questions)

# Save the synthetic data to a JSONL file
with open('synthetic_data.jsonl', 'w') as f:
    for entry in synthetic_data:
        f.write(json.dumps(entry) + '\n')

## Upload Files to S3 for Training Job

In [11]:
# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = 'synthetic-data-gen-workshop'  # Create a new bucket or use an existing one
subdirectory = 'llama-405b-synthetic-training-data'
train_data_location = f"s3://{bucket_name}/{subdirectory}"

files_to_upload = ['template.json','synthetic_data_gen_faq_file.jsonl']

# Upload the files to the specified subdirectory
for file_name in files_to_upload:
    file_path = file_name  # File is in the same directory as the notebook
    key_path = f"{subdirectory}/{file_name}"
    
    # Check if the file exists
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"No such file or directory: '{file_path}'")
    
    # Upload the file
    try:
        s3.upload_file(file_path, bucket_name, key_path)
        print(f"File {file_name} uploaded successfully to {key_path}.")
    except ClientError as e:
        print(f"Error uploading file {file_name}: {e}")

File template.json uploaded successfully to llama-405b-synthetic-training-data/template.json.
File synthetic_data_gen_faq_file.jsonl uploaded successfully to llama-405b-synthetic-training-data/synthetic_data_gen_faq_file.jsonl.


## Distillation by Fine-tuning Llama 3 8B

In this section, we will dive deep into the process of distillation by fine-tuning the LLama 3 8B model to enhance its performance for specific tasks. Fine-tuning involves training the pre-trained model on custom datasets to adapt it to particular domains or applications. This process can be resource-intensive, but using techniques such as LoRA (Low Rank Adaptation) and QLoRA (Quantized LoRA) can significantly reduce the required computational resources and costs. We will explore how to set up and execute a fine-tuning job using SageMaker.

> You'll need a `g5.12xlarge` instance for endpoint usage to deploy this model.

In [12]:
from sagemaker.jumpstart.estimator import JumpStartEstimator

model_id, model_version = "meta-textgeneration-llama-3-8b-instruct", "*"


estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    environment={"accept_eula": "true"},  # Please change {"accept_eula": "true"}
    disable_output_compression=True,
    instance_type="ml.g5.12xlarge",  # For Llama-3-70b, add instance_type = "ml.g5.48xlarge"
)
# By default, instruction tuning is set to false. Thus, to use instruction tuning dataset you use
estimator.set_hyperparameters(
    instruction_tuned="True", epoch="2", max_input_length="1024", chat_dataset="False"
)
estimator.fit({"training": train_data_location})

[2024-08-06 07:39:47,952] p8434 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
[2024-08-06 07:39:48,289] p8434 {session.py:1036} INFO - Creating training-job with name: meta-textgeneration-llama-3-8b-instruct-2024-08-06-07-39-47-904


2024-08-06 07:39:48 Starting - Starting the training job...
2024-08-06 07:40:05 Pending - Training job waiting for capacity...
2024-08-06 07:40:30 Pending - Preparing the instances for training...
2024-08-06 07:41:07 Downloading - Downloading input data...........................
2024-08-06 07:45:40 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-08-06 07:45:43,369 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-08-06 07:45:43,405 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-06 07:45:43,415 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-08-06 07:45:43,417 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-08-06 07:45:52,562 sagemaker-training-toolkit INFO     Installing depend

## Testing the LLama 3 8B Fine-tuned Model 

In this section, we will evaluate the performance of the fine-tuned LLaMA 3 8B model to determine how well it has adapted to the specific tasks for which it was trained. Testing involves comparing the model's responses to a set of predefined questions or tasks against the baseline performance of the original, pre-trained model. This process helps us understand the improvements achieved through distillation by fine-tuning and identify any remaining areas for enhancement. By systematically examining the model's outputs, we can ensure that the fine-tuning process has effectively tailored the model to meet our specific requirements.

In [13]:
finetuned_predictor = estimator.deploy()

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.12xlarge.
[2024-08-06 08:45:45,418] p8434 {model.py:237} INFO - No instance type selected for inference hosting endpoint. Defaulting to ml.g5.12xlarge.
[2024-08-06 08:45:45,508] p8434 {session.py:3961} INFO - Creating model with name: meta-textgeneration-llama-3-8b-instruct-2024-08-06-08-45-45-423
[2024-08-06 08:45:46,175] p8434 {session.py:5725} INFO - Creating endpoint-config with name meta-textgeneration-llama-3-8b-instruct-2024-08-06-08-45-45-417
[2024-08-06 08:45:46,450] p8434 {session.py:4571} INFO - Creating endpoint with name meta-textgeneration-llama-3-8b-instruct-2024-08-06-08-45-45-417


-------------!

In [16]:
# Extract 8 questions, options, and their correct answers from the dataset
# Extract questions, options, and correct answers from the dataset
# Extract 8 questions and their correct answers from the test split
num_questions = 8
questions = dataset['test'].select(range(num_questions))['question']
answers = dataset['test'].select(range(num_questions))['answer']

# Define the inference parameters
params = {
    "max_new_tokens": 512,  # Increase this value to allow longer responses
    "top_p": 0.9,  # Adjust to introduce variability
    "temperature": 0.0,  # Adjust to introduce variability
    "details": True,
    "stop": "<|eot_id|>"
}

# Define the example payloads list
example_payloads = [
    {
        "inputs": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "parameters": params
    }
    for question in questions
]

# Function to run inference and collect results
def run_inference(predictor, example_payloads):
    results = []
    for payload in example_payloads:
        response = predictor.predict(payload)
        response = response[0] if isinstance(response, list) else response
        generated_text = response["generated_text"].strip()
        
        # Check if the response is truncated
        if generated_text.endswith("..."):
            generated_text += " [TRUNCATED]"
        
        results.append(generated_text)
    return results

# Run inference with both models
print("Running inference with fine-tuned LLama 3 8B model...\n")
results_fine_tuned_8b = run_inference(finetuned_predictor, example_payloads)

print("Running inference with LLama 3 8B model...\n")
results_8b = run_inference(llama_3_8b_predictor, example_payloads)


# Create a table of the outputs using HTML
table_html = """
<table>
    <tr>
        <th>Question</th>
        <th>Dataset Answer</th>
        <th>Fine-tuned LLama 3 8B Output</th>
        <th>LLama 3 8B Output</th>
    </tr>
"""

for i in range(8):
    table_html += f"""
    <tr>
        <td>{questions[i]}</td>
        <td>{answers[i]}</td>
        <td>{results_fine_tuned_8b[i]}</td>
        <td>{results_8b[i]}</td>
    </tr>
    """

# Display the table using HTML
display(HTML(table_html))

Running inference with fine-tuned LLama 3 8B model...

Running inference with LLama 3 8B model...



Question,Dataset Answer,Fine-tuned LLama 3 8B Output,LLama 3 8B Output
"If I have patients with asthma, do I need to make any changes to their daily asthma preventive management regimens to reduce their risk of getting sick with COVID-19?","People with moderate to severe asthma, particularly if not well controlled, might be at higher risk of getting very sick from COVID-19. Based on what we currently know about COVID-19, the selection of therapeutic options through guideline-recommended treatment of asthma has not been affected. National asthma guidelines are available. Continuation of inhaled corticosteroids is particularly important for patients already using these medications because there is no evidence of increased risk of COVID-19 morbidity with use of inhaled corticosteroids and an abundance of data showing reduced risk of asthma exacerbation with maintenance of asthma controller therapy. Patients with asthma but without symptoms or a diagnosis of COVID-19 should continue any required nebulizer treatments.","Yes, patients with asthma should take steps to reduce their risk of getting sick with COVID-19. Here are some recommendations for modifying their daily asthma preventive management regimens: 1. **Continue regular asthma medications**: Patients with asthma should continue taking their prescribed medications as directed by their healthcare provider. This includes inhaled corticosteroids, long-acting beta agonists, and other medications to control their asthma symptoms. 2. **Adjust medication dosing**: If a patient's asthma symptoms worsen or they experience increased respiratory symptoms, such as cough, wheezing, or shortness of breath, they should contact their healthcare provider to adjust their medication dosing or add additional medications as needed. 3. **Use inhalers correctly**: Patients should use their inhalers correctly, including: * Using a spacer device with metered-dose inhalers (MDIs) to help deliver medication deeper into the lungs. * Inhaling slowly and deeply through the mouthpiece of the inhaler. * Holding their breath for 1-2 seconds after inhaling. * Exhaling slowly and naturally. 4. **Monitor peak flow**: Patients should monitor their peak flow readings regularly to track their lung function and adjust their medication as needed. 5. **Avoid triggers**: Patients should avoid triggers that can exacerbate asthma symptoms, such as: * Allergens (e.g., pollen, dust mites, pet dander). * Irritants (e.g., tobacco smoke, strong odors, chemicals). * Respiratory viruses (e.g., rhinovirus, coronavirus). 6. **Stay hydrated**: Drinking plenty of fluids can help thin mucus and make it easier to breathe. 7. **Get vaccinated**: Patients with asthma should receive the COVID-19 vaccine, as recommended by the Centers for Disease Control and Prevention (CDC) and the American Lung Association. 8. **Wear a mask**: Patients with asthma should wear a mask when in public or around others to reduce the risk of exposure to respiratory viruses. 9. **Stay informed**: Patients should stay informed about COVID-19 and follow local health guidelines and recommendations. 10. **Contact healthcare provider**: Patients should contact their healthcare provider if they experience any changes in their asthma symptoms, such as increased wheezing, coughing, or shortness of breath, or if they have concerns about their asthma management during the pandemic. By following these guidelines, patients with asthma can reduce their risk of getting sick with COVID-19 and manage their asthma symptoms effectively","Yes, patients with asthma should take steps to reduce their risk of getting sick with COVID-19, and their daily asthma preventive management regimens may need to be adjusted accordingly. Here are some recommendations: 1. **Continue regular asthma medications**: Patients with asthma should continue taking their prescribed medications as directed, including inhaled corticosteroids, long-acting beta2-agonists, and other medications as prescrib

## Conclusion

In this notebook, we have successfully demonstrated the process of distillation by fine-tuning and evaluating the LLama 3 8B model using Amazon SageMaker JumpStart. By leveraging the advanced capabilities of the LLama 3.1 405B model, we generated high-quality synthetic data that served as a foundation for fine-tuning the smaller 8B model. This approach allowed us to enhance the performance of the Llama 3 8B model, tailoring it to specific domain tasks and improving its accuracy and effectiveness.

### Key Steps Accomplished:
1. **Dataset Exploration**: We explored a sample dataset to understand its structure and contents, preparing it for use in model training and evaluation.
2. **Data Generation with LLama 3.1 405B**: Utilizing the LLama 3.1 405B model, we generated synthetic data that provided high-quality responses to domain-specific prompts.
3. **Distillation by Fine-Tuning LLama 3 8B**: We fine-tuned the LLaMA 3 8B model using the synthetic data, adapting it to better handle specific tasks and improving its overall performance.
4. **Model Testing**: We tested the fine-tuned model against a set of evaluation questions, comparing its responses to those of the pre-trained model and assessing the improvements achieved through distillation by fine-tuning.

### Results and Insights:
- **Enhanced Performance**: The fine-tuned LLama 3 8B model demonstrated significant improvements in generating accurate and contextually relevant responses, showcasing the effectiveness of the fine-tuning process.
- **Cost-Effective Adaptation**: By fine-tuning the smaller 8B model with data generated from the larger 405B model, we achieved high performance without the need for extensive computational resources, highlighting a cost-effective approach to model adaptation.
- **Scalability and Flexibility**: The workflow outlined in this notebook can be scaled and adapted to various domains and tasks, providing a flexible framework for enhancing the capabilities of language models.

### Future Work:
- **Further Fine-Tuning**: Additional fine-tuning with more diverse and extensive datasets can further improve the model's performance and adaptability to different domains.
- **Real-World Applications**: Deploying the fine-tuned model in real-world applications such as customer support, content generation, and domain-specific research can provide valuable insights and practical benefits.
- **Continuous Evaluation**: Ongoing evaluation and monitoring of the model's performance will ensure that it remains effective and relevant as new data and requirements emerge.

In conclusion, this notebook has provided a comprehensive guide to generate synthetic data using Llama 3.1 405B and use the generated data for distillation by fine-tuning and evaluating the LLama 3 8B model, demonstrating the potential of using advanced language models to address specific domain needs. By the steps outlined, practitioners can enhance their models' performance, achieve cost-effective adaptations, and unlock new possibilities in natural language processing and beyond.

In [ ]:
# llama_3_8b_predictor.delete_predictor()

# llama_3_1_405b_predictor.delete_predictor()

# finetuned_predictor.delete_predictor()
